In [1]:
!pip install datasets
!pip install transformers 
!pip install evaluate
!pip install rouge-score
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 10.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.7 MB/s eta 0:00

In [2]:
from pathlib import Path
from sklearn.model_selection import train_test_split
from google.colab import drive
from transformers import pipeline
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

import datasets
import re
import time
import math
import evaluate
import torch
import numpy as np
import pandas as pd

# Read in files and subset

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%ls

gdrive/  sample_data/


In [5]:
# read in text and summaries
all_text = pd.read_csv("/content/gdrive/MyDrive/modified_data/text_and_summaries_filtered_split.csv")
all_text.head(-20)

,state_name,state,bill_id,bill_name,original_text,cleaned_text,split_cleaned_text,summary,summary_source,category,status,link,token_list,token_list_lens,split_idx_list,split_text_512,doc_number,split_text
0,Alaska,AK,AK HB27,HB27,HB0027a 1 HB 27 New Text Underlined DEL...,. section . as . is amended by adding new sub...,"['section', 'as', 'is amended by adding new s...",The bill amends an existing regulation of scho...,handwritten,Schools & Education,"Referred to committee, 01/19/2023",https://www.akleg.gov/basis/Bill/Detail/33?Roo...,"[[1375, 1], [38, 1], [19, 21012, 57, 2651, 126...","[1, 1, 8, 11, 1, 1, 8, 1, 6, 1, 8, 12, 9, 7, 8...",[],"['section', 'as', 'is amended by adding new s...",1,section as is amended by adding new subsectio...
1,Alaska,AK,AK HB105,HB105,HB0105a 1 HB 105 New Text Underlined DE...,. section . as . is amended to read . local...,"['section', 'as', 'is amended to read', 'local...",This bill adds to the list of parental rights ...,fast democracy,Schools & Education,"First read and referred to committee, 03/08/2023",https://www.akleg.gov/basis/Bill/Detail/33?Roo...,"[[1375, 1], [38, 1], [19, 21012, 12, 608, 1], ...","[1, 1, 4, 10, 12, 9, 11, 15, 2, 11, 12, 3, 8, ...","[61, 117]","['section', 'as', 'is amended to read', 'local...",1,section as is amended to read local school boa...
2,Alaska,AK,AK HB105,HB105,HB0105a 1 HB 105 New Text Underlined DE...,. section . as . is amended to read . local...,"['section', 'as', 'is amended to read', 'local...",This bill adds to the list of parental rights ...,fast democracy,Schools & Education,"First read and referred to committee, 03/08/2023",https://www.akleg.gov/basis/Bill/Detail/33?Roo...,"[[1375, 1], [38, 1], [19, 21012, 12, 608, 1], ...","[1, 1, 4, 10, 12, 9, 11, 15, 2, 11, 12, 3, 8, ...","[61, 117]",['access to school records by parent foster pa...,2,access to school records by parent foster pare...
3,Alaska,AK,AK HB105,HB105,HB0105a 1 HB 105 New Text Underlined DE...,. section . as . is amended to read . local...,"['section', 'as', 'is amended to read', 'local...",This bill adds to the list of parental rights ...,fast democracy,Schools & Education,"First read and referred to committee, 03/08/2023",https://www.akleg.gov/basis/Bill/Detail/33?Roo...,"[[1375, 1], [38, 1], [19, 21012, 12, 608, 1], ...","[1, 1, 4, 10, 12, 9, 11, 15, 2, 11, 12, 3, 8, ...","[61, 117]",['including the nonviolent resolution or media...,3,including the nonviolent resolution or mediati...
4,Alaska,AK,AK SB96,SB96,SB0096A 1 SB 96 New Text Underlined DEL...,. section . as . is amended to read . local...,"['section', 'as', 'is amended to read', 'local...",This bill adds to the list of parental rights ...,fast democracy,Schools & Education,"First read and referred to committee, 03/08/2023",https://www.akleg.gov/basis/Bill/Detail/33?Roo...,"[[1375, 1], [38, 1], [19, 21012, 12, 608, 1], ...","[1, 1, 4, 10, 12, 9, 11, 15, 2, 11, 12, 2, 8, ...","[62, 118]","['section', 'as', 'is amended to read', 'local...",1,section as is amended to read local school boa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616,West Virginia,WV,WV SB697,SB697,W E S T V I R G I N I A L E G I S L A T U R ...,. article . the health and natural development...,"['article', 'the health and natural developmen...","This bill prohibits gender affirming care, and...",ACLU WV,Healthcare,"Referred to committee, 02/20/2023",http://www.wvlegislature.gov/Bill_Status/Bills...,"[[1108, 1], [8, 533, 11, 793, 606, 1711, 1810,...","[1, 7, 2, 33, 18, 112, 22, 3, 4, 221, 2, 68, 4...","[12, 25, 39, 56]",['performing surgeries that sterilize includin...,2,performing surgeries that sterilize including ...
1617,West Virginia,WV,WV SB697,SB697,W E S T V I R G I N I A L E G I S L A T U R ...,. article . the health and natural development...,"['article', 'the health and natural developmen...","This bill prohibits gender affirming care, and...",ACLU WV,Healthcare,"Referred to committee, 02/20/2023",http://www.wvlegi

In [6]:
cols_keep = ['state','bill_id', 'summary', 'doc_number', 'split_text']

all_text = all_text[cols_keep]
all_text.rename({'bill_id':'title',
                 'split_text':'text',
                }, axis='columns', inplace=True)
all_text.head()

,state,title,summary,doc_number,text
0,AK,AK HB27,The bill amends an existing regulation of scho...,1,section as is amended by adding new subsectio...
1,AK,AK HB105,This bill adds to the list of parental rights ...,1,section as is amended to read local school boa...
2,AK,AK HB105,This bill adds to the list of parental rights ...,2,access to school records by parent foster pare...
3,AK,AK HB105,This bill adds to the list of parental rights ...,3,including the nonviolent resolution or mediati...
4,AK,AK SB96,This bill adds to the list of parental rights ...,1,section as is amended to read local school boa...


## Split into training and testing

In [9]:
train, test = train_test_split(all_text[['text', 'summary', 'title']], test_size = 0.2)
print(len(train))
print(len(test))


1312
329


In [11]:
test

,text,summary,title
1091,section this acts hall be known and maybe cite...,"THIS BILL ENACTS THE ""FAMILIES' RIGHTS AND RES...",SC H3485
1554,other adverse action g any public elementary o...,"This bill requires each interscholastic, inter...",VA HB1399
843,the department of elementary and secondary edu...,"This bill defines ""sex"" as, the two main categ...",MO HB183
134,any other necessary aspects for the operation ...,This bill is a comprehensive education reform ...,AR SB294
1017,an institution that suffers director indirect ...,This bill relates to requiring institutions of...,ND HB1489
...,...,...,...
1353,shall establish the prior authorization requir...,This bill recodifies provisions regarding heal...,UT SB39
590,dj kw st families rights and resp on sib li ti...,"This bill establishes the ""families' rights an...",MS HB509
1471,utah medical examiner the medical examiners ha...,This bill recodifies provisions regarding heal...,UT SB39
215,jointly appoint faculty committees to review a...,Building on the work the Legislature has under...,FL HB999


In [13]:
train.to_csv("/content/gdrive/MyDrive/modified_data/train_bills.csv")
test.to_csv("/content/gdrive/MyDrive/modified_data/test_bills.csv")

# Create HuggingFace Objects

In [ ]:
train_dataset = datasets.Dataset.from_dict(train)
test_dataset = datasets.Dataset.from_dict(test)
billsum = datasets.DatasetDict({"train": train_dataset, "test": test_dataset})

# Data Loading

In [ ]:
billsum['test'][0]['summary']

'This bill prohibits a healthcare provider from performing on a minor or administering to a minor a medical procedure if the performance or administration of the procedure is for the purpose of enabling a minor to identify with, or live as, a purported identity inconsistent with the minor’s sex.'

# Load Tokenizer

In [ ]:
## PREPROCESSING
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
#The preprocessing function you want to create needs to:

# 1. Prefix the input with a prompt so T5 knows this is a summarization task. 
#    Some models capable of multiple NLP tasks require prompting for specific tasks.
# 2. Use the keyword text_target argument when tokenizing labels.
# 3. Truncate sequences to be no longer than the maximum length set by the max_length parameter.

prefix = "summarize: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    # Tokenizes and truncates
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# To apply the preprocessing function over the entire dataset
# You can speed up the map function by setting batched=True to process multiple elements of the dataset at once
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/1312 [00:00<?, ? examples/s]

Map:   0%|          | 0/329 [00:00<?, ? examples/s]

In [ ]:
tokenized_billsum['train']

Dataset({
    features: ['text', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1312
})

In [ ]:
# Create batch of examples using DataCollatorForSeq2Seq. It's better to dynamically pad the sentences to the longest length
# in a batch during collation instead of padding to the model's maximum length
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

# Set Up Evaluation

In [ ]:
## SET UP EVALUATOR

# Load the ROUGE metric
rouge = evaluate.load("rouge")

# Create function that passes predictions and labels to compute the ROUGE metric
# This is what is used for model training
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# Train Model

In [ ]:
#model.save_weights("/content/gdrive/My Drive/weights.h5")

In [ ]:
%ls

gdrive/  sample_data/


In [ ]:
output_directory = "/content/gdrive/My Drive/modified_data"
model_directory =  "/content/gdrive/My Drive/summarizer_model"

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

training_args = Seq2SeqTrainingArguments(
#     output_dir="/content/gdrive/My Drive/my_awesome_billsum_model",
    output_dir = output_directory,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
#     fp16=True,
    push_to_hub=False, # changed to false
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
start = time.time()
trainer.train()
end = time.time()
print((end - start)/60)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.318386,0.127000,0.023100,0.101500,0.101900,18.997000
2,No log,3.071294,0.135800,0.025500,0.108800,0.108800,19.000000
3,No log,2.973733,0.161300,0.042000,0.131900,0.131600,18.987800
4,No log,2.946281,0.174100,0.050900,0.143700,0.143500,18.960500


4.6460633436838785


In [ ]:
# Save the model
trainer.save_model(model_directory)

# Test on all bills

In [ ]:
all_text

,state,title,summary,doc_number,text
0,AK,AK HB27,The bill amends an existing regulation of scho...,1,section as is amended by adding new subsectio...
1,AK,AK HB105,This bill adds to the list of parental rights ...,1,section as is amended to read local school boa...
2,AK,AK HB105,This bill adds to the list of parental rights ...,2,access to school records by parent foster pare...
3,AK,AK HB105,This bill adds to the list of parental rights ...,3,including the nonviolent resolution or mediati...
4,AK,AK SB96,This bill adds to the list of parental rights ...,1,section as is amended to read local school boa...
...,...,...,...,...,...
1636,WY,WY SF133,This bill prohibits students of the male sex f...,6,ii the commission may request additional evide...
1637,WY,WY SF133,This bill prohibits students of the male sex f...,7,days after the day on which the commission ren...
1638,WY,WY SF144,This would prohibit physicians from performing...,1,section ws and are created to read state of wy...
1639,WY,WY SF144,This would prohibit physicians from performing...,2,procedures or treatments that are performed wi...


In [ ]:
unique_bills = all_text.title.unique()
summarizer = pipeline("summarization", model=str(model_directory))

all_summaries = []

for id in unique_bills:
  print("BILL:", id)
  # Filter for the rows of the current bill
  curr_bill_text = all_text[(all_text.title == id)]
  # Get the summary of the current bill
  compiled_summary = []
  for bill_section in list(curr_bill_text.text):
      model_summary = summarizer(bill_section)
      compiled_summary.append(model_summary[0]["summary_text"])
  all_summaries.append(compiled_summary)

# Put all the summaries in a data frame
model_summaries_df = pd.DataFrame()
model_summaries_df["bill_id"] = unique_bills
model_summaries_df["model_summary"] = all_summaries

BILL: AK HB27
BILL: AK HB105


Your max_length is set to 200, but you input_length is only 116. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=58)
Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


BILL: AK SB96


Your max_length is set to 200, but you input_length is only 102. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


BILL: AZ SB1028
BILL: AZ SB1026
BILL: AZ SB1001
BILL: AZ SB1030
BILL: AZ SB1040


...

Your max_length is set to 200, but you input_length is only 114. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)


BILL: WV HB3503
BILL: WV SB692
BILL: WV SB697
BILL: WY HB187
BILL: WY SF111
BILL: WY SF117
BILL: WY HB262


Your max_length is set to 200, but you input_length is only 28. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


BILL: WY SF133
BILL: WY SF144


Your max_length is set to 200, but you input_length is only 155. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=77)


In [ ]:
model_summaries_df.to_csv("modified_data/model_summaries.csv")

OSError: ignored